## Save and restore models
https://www.tensorflow.org/alpha/tutorials/keras/save_and_restore_models

A demonstration of model saving and restoring using the MNIST dataset.

In [1]:
import os

import tensorflow as tf
from tensorflow import keras

### Data

In [2]:
(trainImages, trainLabels), (testImages, testLabels) = keras.datasets.mnist.load_data()

# Use fewer dataset for demonstration.
trainLabels = trainLabels[:1000]
testLabels = testLabels[:1000]

# Truncation and normalization
trainImages = trainImages[:1000].reshape(-1, 28*28) / 255.
testImages = testImages[:1000].reshape(-1, 28*28) / 255.

### Model

In [5]:
def buildModel():
    model = keras.models.Sequential([
        keras.layers.Dense(512, activation='relu',
                           input_shape=(trainImages.shape[1],)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [6]:
model = buildModel()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### Save checkpoints during training
We'll use `tf.keras.callbacks.ModelCheckpoint` to save models during and at the end of training.

In [12]:
checkpointPath = 'model-saving-example/cp.ckpt'

# Create a checkpoint callback.
cpCallback = keras.callbacks.ModelCheckpoint(checkpointPath,
                                             save_weights_only=True,
                                             verbose=1)

# Train with the callback.
model = buildModel()
model.fit(trainImages, trainLabels,
          epochs=10,
          validation_data=(testImages, testLabels),
          callbacks=[cpCallback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 416/1000 [===========>..................] - ETA: 0s - loss: 1.7395 - accuracy: 0.5096
Epoch 00001: saving model to model-saving-example/cp.ckpt
1000/1000 [==============================] - 0s 307us/sample - loss: 1.1766 - accuracy: 0.6740 - val_loss: 0.7011 - val_accuracy: 0.7740
Epoch 2/10
 576/1000 [================>.............] - ETA: 0s - loss: 0.4633 - accuracy: 0.8715
Epoch 00002: saving model to model-saving-example/cp.ckpt
1000/1000 [==============================] - 0s 136us/sample - loss: 0.4378 - accuracy: 0.8820 - val_loss: 0.5388 - val_accuracy: 0.8310
Epoch 3/10
 640/1000 [==================>...........] - ETA: 0s - loss: 0.2822 - accuracy: 0.9203
Epoch 00003: saving model to model-saving-example/cp.ckpt
1000/1000 [==============================] - 0s 132us/sample - loss: 0.3017 - accuracy: 0.9190 - val_loss: 0.4596 - val_accuracy: 0.8630
Epoch 4/10
 672/1000 [===================>..........] - ETA: 0s - loss: 0

We can see a checkpoint file is created.

In [20]:
!ls -lh $(dirname {checkpointPath})

total 4.2M
-rw-rw-rw- 1 yeon yeon   71 Jun  4 11:12 checkpoint
-rw-rw-rw- 1 yeon yeon 4.7M Jun  4 11:12 cp.ckpt.data-00000-of-00001
-rw-rw-rw- 1 yeon yeon 1.6K Jun  4 11:12 cp.ckpt.index


### Load the checkpoint
Let's build a fresh model and evaluate it right away.

In [21]:
model = buildModel()
loss, accuracy = model.evaluate(testImages, testLabels)
print(f"Untrained model accuracy: {accuracy:5.2%}")

1000/1000 [==============================] - 0s 66us/sample - loss: 2.3579 - accuracy: 0.0970
Untrained model accuracy: 9.70%


Let's load the saved weights and re-evaluate the model.

In [22]:
model.load_weights(checkpointPath)
loss, accuracy = model.evaluate(testImages, testLabels)
print(f"Restored model accuracy: {accuracy:5.2%}")

1000/1000 [==============================] - 0s 25us/sample - loss: 0.3973 - accuracy: 0.8730
Restored model accuracy: 87.30%


### Checkpoint callback options
Let's give a custom indicator to checkpoint files and use different saving options.

In [23]:
checkpointPath = 'model-saving-example/cp-{epoch:04d}.ckpt'
cpCallback = keras.callbacks.ModelCheckpoint(
    checkpointPath, verbose=1, save_weights_only=True,
    # Save weights every 5-epochs.
    period=5)

model = buildModel()
model.save_weights(checkpointPath.format(epoch=0))
model.fit(trainImages, trainLabels, epochs=50,
          validation_data=(testImages, testLabels),
          callbacks=[cpCallback],
          verbose=0)


Epoch 00005: saving model to model-saving-example/cp-0005.ckpt

Epoch 00010: saving model to model-saving-example/cp-0010.ckpt

Epoch 00015: saving model to model-saving-example/cp-0015.ckpt

Epoch 00020: saving model to model-saving-example/cp-0020.ckpt

Epoch 00025: saving model to model-saving-example/cp-0025.ckpt

Epoch 00030: saving model to model-saving-example/cp-0030.ckpt

Epoch 00035: saving model to model-saving-example/cp-0035.ckpt

Epoch 00040: saving model to model-saving-example/cp-0040.ckpt

Epoch 00045: saving model to model-saving-example/cp-0045.ckpt

Epoch 00050: saving model to model-saving-example/cp-0050.ckpt


We can see the newly saved checkpoints:

In [29]:
!ls -lhtr $(dirname {checkpointPath})

total 47M
-rw-rw-rw- 1 yeon yeon 4.7M Jun  4 11:12 cp.ckpt.data-00000-of-00001
-rw-rw-rw- 1 yeon yeon 1.6K Jun  4 11:12 cp.ckpt.index
-rw-rw-rw- 1 yeon yeon 1.6M Jun  4 11:26 cp-0000.ckpt.data-00000-of-00001
-rw-rw-rw- 1 yeon yeon  706 Jun  4 11:26 cp-0000.ckpt.index
-rw-rw-rw- 1 yeon yeon 4.7M Jun  4 11:26 cp-0005.ckpt.data-00000-of-00001
-rw-rw-rw- 1 yeon yeon 1.6K Jun  4 11:26 cp-0005.ckpt.index
-rw-rw-rw- 1 yeon yeon 4.7M Jun  4 11:26 cp-0010.ckpt.data-00000-of-00001
-rw-rw-rw- 1 yeon yeon 1.6K Jun  4 11:26 cp-0010.ckpt.index
-rw-rw-rw- 1 yeon yeon 1.6K Jun  4 11:26 cp-0015.ckpt.index
-rw-rw-rw- 1 yeon yeon 4.7M Jun  4 11:26 cp-0015.ckpt.data-00000-of-00001
-rw-rw-rw- 1 yeon yeon 1.6K Jun  4 11:26 cp-0020.ckpt.index
-rw-rw-rw- 1 yeon yeon 4.7M Jun  4 11:26 cp-0020.ckpt.data-00000-of-00001
-rw-rw-rw- 1 yeon yeon 4.7M Jun  4 11:26 cp-0025.ckpt.data-00000-of-00001
-rw-rw-rw- 1 yeon yeon 1.6K Jun  4 11:26 cp-0025.ckpt.index
-rw-rw-rw- 1 yeon yeon 4.7M Jun  4 11:26 cp-0030.ckpt.data-000

from which we can choose the latest one:

In [30]:
latestCheckpoint = tf.train.latest_checkpoint(os.path.dirname(checkpointPath))
print(latestCheckpoint)

model-saving-example/cp-0050.ckpt


Let's load and evaluate the latest weights.

In [31]:
model = buildModel()
model.load_weights(latestCheckpoint)
loss, accuracy = model.evaluate(testImages, testLabels)
print(f"Restored model accuracy: {accuracy:5.2%}")

1000/1000 [==============================] - 0s 69us/sample - loss: 0.4752 - accuracy: 0.8780
Restored model accuracy: 87.80%


Checkpoint saving consists of
- One or more shards containing the model's weights, and
- An index file indicating which weights are in which shard.

When training a model on a single machine, only one shard with suffix `.data-00000-of-00001` is saved.

### Manually save weights
Model weights can be manually saved.

In [33]:
model.save_weights('model-saving-example/manual_checkpoint')

Note the checkpoint names:

In [35]:
!ls -lhtr model-saving-example | tail

-rw-rw-rw- 1 yeon yeon 1.6K Jun  4 11:26 cp-0035.ckpt.index
-rw-rw-rw- 1 yeon yeon 4.7M Jun  4 11:26 cp-0040.ckpt.data-00000-of-00001
-rw-rw-rw- 1 yeon yeon 1.6K Jun  4 11:26 cp-0040.ckpt.index
-rw-rw-rw- 1 yeon yeon 1.6K Jun  4 11:26 cp-0045.ckpt.index
-rw-rw-rw- 1 yeon yeon 4.7M Jun  4 11:26 cp-0045.ckpt.data-00000-of-00001
-rw-rw-rw- 1 yeon yeon 1.6K Jun  4 11:26 cp-0050.ckpt.index
-rw-rw-rw- 1 yeon yeon 4.7M Jun  4 11:26 cp-0050.ckpt.data-00000-of-00001
-rw-rw-rw- 1 yeon yeon 4.7M Jun  4 11:46 manual_checkpoint.data-00000-of-00001
-rw-rw-rw- 1 yeon yeon 1.3K Jun  4 11:46 manual_checkpoint.index
-rw-rw-rw- 1 yeon yeon   91 Jun  4 11:46 checkpoint


Test the weights.

In [36]:
model = buildModel()
model.load_weights('model-saving-example/manual_checkpoint')
loss, accuracy = model.evaluate(testImages, testLabels)
print(f"Restored model accuracy: {accuracy:5.2%}")

1000/1000 [==============================] - 0s 62us/sample - loss: 0.4752 - accuracy: 0.8780
Restored model accuracy: 87.80%


### Save an entire model
A model and its optimizer can be saved to a file.

Let's first clean the save directory.

In [37]:
!rm model-saving-example/*

#### As an HDF5

In [38]:
model = buildModel()
model.fit(trainImages, trainLabels, epochs=5, verbose=0)
model.save('model-saving-example/my_model.h5')

In [40]:
!ls -lh model-saving-example/

total 7.0M
-rw-rw-rw- 1 yeon yeon 4.7M Jun  4 11:53 my_model.h5


Build a model directly from the file:

In [41]:
newModel = keras.models.load_model('model-saving-example/my_model.h5')
newModel.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Check the accuracy.

In [42]:
loss, accuracy = newModel.evaluate(testImages, testLabels)
print(f"Restored model accuracy: {accuracy:5.2%}")

1000/1000 [==============================] - 0s 68us/sample - loss: 0.4233 - accuracy: 0.8620
Restored model accuracy: 86.20%


Note that currently it is *not* possible to save **Tensorflow** optimizers (from `tf.train`). If Tensorflow optimizers are needed, we need to re-compile the model after loading; the state of the optimizer has to be lost.

#### As a `saved_model`
This method is still experimental for `tf.keras` models.

Let's save the model with a timestamp.

In [43]:
import time
savedModelPath = f'model-saving-example/{time.time()}'
keras.experimental.export_saved_model(model, savedModelPath)

W0604 12:03:07.828806 139753968699200 deprecation.py:323] From /home/yeon/appl/anaconda3/envs/ml/lib/python3.7/site-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:253: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
W0604 12:03:07.829826 139753968699200 tf_logging.py:161] Export includes no default signature!
W0604 12:03:08.141908 139753968699200 tf_logging.py:161] Export includes no default signature!


In [45]:
!ls -lhtr model-saving-example/

total 7.0M
-rw-rw-rw- 1 yeon yeon 4.7M Jun  4 11:53 my_model.h5
drwxr-xr-x 1 yeon yeon 4.0K Jun  4 12:03 1559617387.3388798


Reload the model:

In [46]:
newModel = keras.experimental.load_from_saved_model(savedModelPath)
newModel.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


The loaded model can be directly used for prediction:

In [50]:
newModel.predict(testImages).shape

(1000, 10)

But for evaluation, we must re-compile the model.

In [51]:
newModel.compile(optimizer=model.optimizer,  # Reuse the original optimizer.
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
loss, accuracy = newModel.evaluate(testImages, testLabels)
print(f"Restored model accuracy: {accuracy:5.2%}")

1000/1000 [==============================] - 0s 65us/sample - loss: 0.4233 - accuracy: 0.8620
Restored model accuracy: 86.20%
